In [1]:
!pip install psycopg2 langchain openai

In [2]:
import psycopg2

# データベース接続情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# PostGISへの接続
connection = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

# クエリの実行
cursor = connection.cursor()

# getTableAndColumnsName
schema = "public"
query = f"""
SELECT
  t.table_name,
  c.*
FROM
  information_schema.tables t
    JOIN information_schema.columns c
      ON t.table_name = c.table_name
WHERE
  t.table_schema = '{schema}'
    AND c.table_schema = '{schema}'
ORDER BY
  t.table_name,
  c.ordinal_position;
"""

# クエリの実行と結果の取得
cursor.execute(query)
results = cursor.fetchall()

# 結果の表示
table_info = []
for row in results:
    table_name = row[3]
    if table_name == "geography_columns":
        continue
    if table_name == "geometry_columns":
        continue
    if table_name == "spatial_ref_sys":
        continue
    column_name = row[4]
    nullable = row[7]
    data_type = row[8]
    data_type_sub = row[28]
    table_info_row = f"table_name: {table_name}, column_name: {column_name}, data_type: {data_type}/{data_type_sub}, nullable: {nullable}"
    table_info.append(table_info_row)

# コネクションのクローズ
cursor.close()
connection.close()

print("\n".join(table_info))

table_name: flood_areas_akita_2023_07_18, column_name: gridcode, data_type: bigint/int8, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: flood_depth, data_type: text/text, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: flood_areas_akita_2023_07_19, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: id, data_type: bigint/int8, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name_en, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: geom, data_type: USER-DEFINED/geometry, nullable: YES
table_name: parks, column_name: id, data_type: bigint/int8, nullable: YES
table_name: parks, column_name: name, data_type: character varying

In [15]:
# PostGISへの接続
connection = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

# クエリの実行
cursor = connection.cursor()

sql_query = """
SELECT row_to_json(featurecollection) FROM (
  SELECT
    'FeatureCollection' AS type,
    array_to_json(array_agg(feature)) AS features
  FROM (
    SELECT
      'Feature' AS type,
      ST_AsGeoJSON(geom)::json AS geometry,
      row_to_json((
        SELECT p FROM (
          SELECT p.id, p.name, p.name_en
        ) AS p
      )) AS properties
    FROM parks_akita_2023_07_18 AS p
    WHERE EXISTS (SELECT 1 FROM flood_areas_akita_2023_07_18 AS f WHERE ST_Intersects(p.geom, f.geometry))
  ) AS feature
) AS featurecollection;
"""

# クエリの実行と結果の取得
cursor.execute(sql_query)
results = cursor.fetchall()

# 結果の表示
import json
print(json.dumps(results[0][0], indent=2))

# コネクションのクローズ
cursor.close()
connection.close()

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              140.1311,
              39.7141341
            ],
            [
              140.1304351,
              39.7141178
            ],
            [
              140.1304055,
              39.7139112
            ],
            [
              140.1303474,
              39.7135103
            ],
            [
              140.1303155,
              39.7132811
            ],
            [
              140.1302182,
              39.7129972
            ],
            [
              140.130341,
              39.7129682
            ],
            [
              140.1303507,
              39.7129954
            ],
            [
              140.1303585,
              39.7130084
            ],
            [
              140.130368,
              39.7130241
            ],
            [
              140

In [18]:
!pip install folium geopandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/cb/7b/2fb6ab376c78d2e60812ad0bf4b24889309d2aeb543f54efcd586c69907f/geopandas-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for fiona>=1.8.19 from https://files.pythonhosted.org/packages/43/56/915f3e4c0016f5917d65f70097e1080f1f5341f3cd2d48d9738c2465fffa/Fiona-1.9.4.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 602.7 kB/s eta 0:00:00 0:00:01
  Obtaining dependency information for pyproj>=3.0.1 from https://files.pythonhosted.org/packages/81/eb/3e31e15fdee9d54bdbc575b6384bd7e54f63590fcb4d5c247ad38a81eb44/pyproj-3.6.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 159.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 350.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [27]:
import folium
import geopandas as gpd

df = gpd.GeoDataFrame.from_features(results[0][0]["features"])
bounds = df.total_bounds.tolist()


m = folium.Map(
    tiles="cartodbpositron",
)

folium.GeoJson(data=results[0][0]).add_to(m)

folium.LayerControl().add_to(m)

m.fit_bounds([bounds[:2][::-1], bounds[2:][::-1]])
m
